<a href="https://colab.research.google.com/github/courtgibson13/UofM-Course-Work-Comprehensive-Project/blob/main/06_Supervised_Ablation_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import, Download, & Variable Statements

In [1]:
! pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.7/535.7 kB 8.0 MB/s eta 0:00:00


In [10]:
!pip install pydantic==1.8.2 typing-extensions==4.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.1
    Uninstalling pydantic-2.6.1:
      Successfully uninstalled pydantic-2.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not instal

In [11]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 22.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [12]:
# Import & download statements
# General Statements
#!git clone https://github.com/d-atallah/implicit_gender_bias.git
#! pip install joblib
#! pip install shap
import pandas as pd
import string
import re
import joblib
#from implicit_gender_bias import config as cf
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import spacy
import shap
import scipy


# Feature selection & Model tuning
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, StratifiedKFold, cross_validate
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD,PCA, NMF
from sklearn.metrics import confusion_matrix,precision_score, recall_score, f1_score, accuracy_score, roc_curve, roc_auc_score, log_loss, make_scorer, average_precision_score

# Model options
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

# NLTK resources
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()

nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Read Inputs

In [19]:
# Variables
folder_path = '/content/drive/MyDrive/Supervised Learning Notebooks/'#'/home/gibsonce/datallah-jaymefis-gibsonce/'

# Load DataFrames from pkl files
X_train = pd.read_pickle(folder_path + 'X_train_preprocessed.pkl')
X_test = pd.read_pickle(folder_path + 'X_test_preprocessed.pkl')
y_train = pd.read_pickle(folder_path + 'y_train.pkl')
y_test = pd.read_pickle(folder_path + 'y_test.pkl')

In [20]:
non_nan_indices_train = ~X_train.isnull()
non_nan_indices_test = ~X_test.isnull()

# Filter y_train and y_test using the non-NaN indices
y_train = y_train[non_nan_indices_train]
y_test = y_test[non_nan_indices_test]

# Filter X_train and X_test to remove NaN records
X_train = X_train[non_nan_indices_train]
X_test = X_test[non_nan_indices_test]

## Define Functions


In [47]:
def tokenize_and_categorize(text):
    # Tokenize and process the text using spaCy
    doc = nlp(text)
    word_features = ' '.join([token.text for token in doc])
    pos_tags = ' '.join([token.pos_ for token in doc])

    return word_features, pos_tags

In [ ]:
def model_testing(X_train, y_train, X_test, y_test, params):
    """
    Runs a specified model and dimensionality reduction method with tuned hyperparameters

    Parameters:
    - X_train (array-like): Training set features, preprocessed.
    - y_train (array-like): Training set labels.
    - X_test (array-like): Test set features, preprocessed.
    - y_test (array-like): Test set labels.
    - model_type (str): Type of model to test. Choose from 'log' (Logistic Regression), 'xgb' (XGBoost), 'knn' (k-Nearest Neighbors), 'svm' (Support Vector Machine).
    - vectorizer (str): Type of vectorizer to test. Choose from 'count' (Count Vecotizer) or 'tfidf' (TF-IDF Vecotizer).
    - ngram (int): Feature representation to test. Choose 1 for unigrams, 2 for bigrams, and so on.
    - params (dict): Hyperparameter grid for the specified model and dimensionality reduction method.

    Returns:
    - Pipeline: Trained and fit pipeline with the best hyperparameters.
    - X_train_ (array-like): Preprocessed  and vectorized training set features.
    - X_test_ (array-like): Preprocessed  and vectorized test set features.
    """
    start_time = time.time()

    X_train_word_features, X_train_pos_tags = zip(*map(tokenize_and_categorize, X_train))
    X_test_word_features, X_test_pos_tags = zip(*map(tokenize_and_categorize, X_test))

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Spacy tokenization completed. Time elapsed: {elapsed_time/60:.2f} minutes.")

    vect = TfidfVectorizer(ngram_range=(1, 1))
    X_train_ = vect.fit_transform(X_train)
    X_test_ = vect.transform(X_test)

    # Initialize a dictionary to store results
    ablation_results = {}

    # Iterate over unique part of speech categories
    unique_pos_tags = np.unique([pos_tag for pos_tags in X_train_pos_tags for pos_tag in pos_tags])
    for pos_tag in unique_pos_tags:
        print(pos_tag)
        # Filter out the current part of speech category
        X_train_filtered = [doc for doc, tags in zip(X_train, X_train_pos_tags) if pos_tag not in tags]
        X_test_filtered = [doc for doc, tags in zip(X_test, X_test_pos_tags) if pos_tag not in tags]

        # Vectorize the word features
        word_features_vectorizer = TfidfVectorizer(ngram_range=(1, 1))
        X_train_word_features = word_features_vectorizer.fit_transform(X_train_filtered)
        X_test_word_features = word_features_vectorizer.transform(X_test_filtered)

        # Vectorize the parts of speech tags
        pos_tags_vectorizer = TfidfVectorizer(ngram_range=(1, 1))
        X_train_pos_tags = pos_tags_vectorizer.fit_transform(X_train_pos_tags)
        X_test_pos_tags = pos_tags_vectorizer.transform(X_test_pos_tags)

        # Combine the vectorized word features and parts of speech tags
        X_train_combined = scipy.sparse.hstack([X_train_word_features, X_train_pos_tags])
        X_test_combined = scipy.sparse.hstack([X_test_word_features, X_test_pos_tags])


        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Vectorization completed. Time elapsed: {elapsed_time/60:.2f} minutes.")

        model = XGBClassifier(random_state=42, **params.get('xgbclassifier', {}))
        model.fit(X_train_combined, y_train)

        feature_importances = model.feature_importances_
        feature_names = word_features_vectorizer.get_feature_names_out()
        feature_importance_dict = dict(zip(feature_names, feature_importances))

        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Pipeline fitting completed. Time elapsed: {elapsed_time/60:.2f} minutes.")

        # Store the result in the dictionary
        ablation_results[pos_tag] = {
            'model': model,
            'train_features': X_train_combined,
            'test_features': X_test_filtered_word_features_,
            'feature_importance_dict': feature_importance_dict
        }





    # Save the trained pipeline
    """joblib.dump(model, f'{folder_path}{model_type}_pipeline.pkl')
    joblib.dump(X_train_, f'{folder_path}{model_type}_X_train.pkl')
    joblib.dump(X_test_, f'{folder_path}{model_type}_X_test.pkl')

    joblib.dump(explainer, f'{folder_path}{model_type}_shap.pkl')
    joblib.dump(feature_importance_dict, f'{folder_path}{model_type}_features.pkl')
    print('Write to pkl file completed.')"""

    return model, X_train_, X_test_, explainer, feature_importance_dict


In [73]:
def model_eval(pipeline, X_test, y_test):
    """
    Evaluates a specified model using accuracy, precision, recall, F-1 score, AUC-ROC & PR, log-Loss, and a confusion matrix.

    Parameters:
    - pipeline (object): Fitted pipeline.
    - X_test (list or array): Test set features.
    - y_test (list or array): True labels.

    Returns:
    - metrics_df (pd.DataFrame): DataFrame containing the metrics and scores.
    - confusion_df (pd.DataFrame): DataFrame containing a confusion matrix.
    """

    start_time = time.time()

    # Cross-validation
    scoring = {
        'f1': make_scorer(f1_score),
        'pr_auc': make_scorer(average_precision_score),
    }
    print('cv scoring created')

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_results = cross_validate(pipeline, X_test, y_test, cv=cv, scoring=scoring)

    print('cv pipeline created')
    # Create DataFrame to store cross-validation results
    cv_metrics_df = pd.DataFrame({
    'Metric': ['F1-Score','AUC-PR'],
    'CV_Mean': [np.mean(cv_results['test_f1']),
                np.mean(cv_results['test_pr_auc']),
                ],
    'CV_Std Dev': [np.std(cv_results['test_f1']),
                   np.std(cv_results['test_pr_auc']),
                  ]
    })

    print("\nCross Validation:")
    print(cv_metrics_df)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Cross validation completed. Time elapsed: {elapsed_time/60:.2f} minutes.")

    return cv_metrics_df

## XGBoost

In [41]:
# Define variables
model = 'xgb'
vectorization = 'tfidf'
ngram = 1
params = {
    'xgbclassifier': {'subsample': 0.8, 'n_estimators': 150, 'max_depth': 9, 'learning_rate': 0.05, 'colsample_bytree': 0.5},
    #'truncatedsvd': {'n_components': 150}
}

# Run model
model, train, test, explainer, feature_importance_dict = model_testing(X_train, y_train, X_test, y_test, model, vectorization, ngram, params)

Spacy tokenization completed. Time elapsed: 2.48 minutes.
Vectorization completed. Time elapsed: 2.49 minutes.
Pipeline fitting completed. Time elapsed: 2.72 minutes.


In [79]:
X_train_word_features, X_train_pos_tags = zip(*map(tokenize_and_categorize, X_train))
X_test_word_features, X_test_pos_tags = zip(*map(tokenize_and_categorize, X_test))

In [ ]:
# Iterate over unique part of speech categories
unique_pos_tags = [
    'ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET', 'INTJ',
    'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X'
]
for pos_tag in unique_pos_tags:
    print(pos_tag)
    # Filter out the current part of speech category
    X_train_filtered = [doc for doc, tags in zip(X_train, X_train_pos_tags) if pos_tag not in tags]
    X_test_filtered = [doc for doc, tags in zip(X_test, X_test_pos_tags) if pos_tag not in tags]

    y_train_filtered = [label for doc, label, tags in zip(X_train, y_train, X_train_pos_tags) if pos_tag not in tags]
    y_test_filtered = [label for doc, label, tags in zip(X_test, y_test, X_test_pos_tags) if pos_tag not in tags]

    # Filter the part of speech tags
    X_train_pos_tags_filtered = [tags for tags in X_train_pos_tags if pos_tag not in tags]
    X_test_pos_tags_filtered = [tags for tags in X_test_pos_tags if pos_tag not in tags]

    # Vectorize the word features
    word_features_vectorizer = TfidfVectorizer(ngram_range=(1, 1))
    X_train_word_features_ = word_features_vectorizer.fit_transform(X_train_filtered)
    X_test_word_features_ = word_features_vectorizer.transform(X_test_filtered)

    # Vectorize the parts of speech tags
    pos_tags_vectorizer = TfidfVectorizer(ngram_range=(1, 1))
    X_train_pos_tags_ = pos_tags_vectorizer.fit_transform(X_train_pos_tags_filtered)
    X_test_pos_tags_ = pos_tags_vectorizer.transform(X_test_pos_tags_filtered)

    # Combine the vectorized word features and parts of speech tags
    X_train_combined = scipy.sparse.hstack([X_train_word_features_, X_train_pos_tags_])
    X_test_combined = scipy.sparse.hstack([X_test_word_features_, X_test_pos_tags_])

    model = XGBClassifier(random_state=42, **params.get('xgbclassifier', {}))
    model.fit(X_train_combined, y_train_filtered)

    metrics_df = model_eval(model, X_test_combined, y_test_filtered)






ADJ
cv scoring created
cv pipeline created

Cross Validation:
     Metric   CV_Mean  CV_Std Dev
0  F1-Score  0.540915    0.031986
1    AUC-PR  0.527738    0.020105
Cross validation completed. Time elapsed: 0.18 minutes.
ADP
cv scoring created


In [81]:
print(X_test_filtered.shape)
print(type(X_test_filtered))

AttributeError: 'list' object has no attribute 'shape'